# Variables

通常使用variable来保存和更新参数。variables是in-memory buffer containing buffers。

他们必须被显式初始化。可以把variable保存到本地，下一次训练时重新读取。

下面介绍两个重要的类：

* tf.Variable
* tf.train.Saver

## Creation

创建Variable时需要传递一个Tensor作为它的初始化值。tensorflow提供了一些列产生Tensor的方法。这些方法有一个共同特点：需要指定tensor的shape。

Variable的shape就是传给他的tensor的shape。

通常，variable的shape是固定的，不过tensorflow也提供了修改variable shape的方法。

In [1]:
import tensorflow as tf

# Create 2 variable 
weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35),
                     name="weights")
biases = tf.Variable(tf.zeros([200]), name="biases")


 ### 指定设备

可以在创建variable时指定设备，只需要用`with tf.device(...)`:

In [ ]:
with tf.device("/cpu:0"):
    v = tf.Variable(...)
    
with tf.device("/gpu:0"):
    v = tf.Variable(...)

with tf.device("/job:ps/task:7"):
    v = tf.Variable(...)

**注意**，对variable进行修改的节点比如`tf.Variable.assign`和`tf.train.Optimizer`必须和variable要在同一个设备。

## Initialization

In [ ]:
# Create two variables.
weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35),
                      name="weights")
biases = tf.Variable(tf.zeros([200]), name="biases")
...

# Add an op to initialize the variables
init_op = tf.global_variables_initializer()

# 
with tf.Session() as sess:
    sess.run(init_op)
    
    # run the model

### 用另一个variable进行初始化

你也可以用另一个variable来对variable进行初始化。

In [2]:
# Create a variable with a random value.
weights = tf.Variable(tf.random_normal([784, 200], stddev=0.35),
                      name="weights")
# 创建另一个variable，它的值和weights相同
w2 = tf.Variable(weights.initialized_value(), name="w2")

w_twice = tf.Variable(weights.initialized_value() * 2.0, name="w_twice")

## Saving和Restoring

最简单的方法是使用`tf.train.Saver`。

### Checkpoint Files

Variable保存到二进制文件，格式是map: {variable_name: tensor_values}。

To understand what variables are in a checkpoint, you can use the inspect_checkpoint library, and in particular, the print_tensors_in_checkpoint_file function.

### 保存variables

In [ ]:
v1 = tf.Variable(..., name="v1")
v2 = tf.Variable(..., name="v2")
...

init_op = tf.global_variables_initializer()

# Add ops to save and restore all the variables
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    ...
    
    # save the variables to disk
    save_path = saver.save(sess, "/tmp/model.ckpt")
    

### Restoring Variables

从文件读取的variable，就不需要再初始化了，注意，这里仅限于所有的variable都是从文件读取到，如果图中有variable不是从文件读取，还需要初始化。

In [ ]:
# Create some variables.
v1 = tf.Variable(..., name="v1")
v2 = tf.Variable(..., name="v2")
...
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.
with tf.Session() as sess:
  # Restore variables from disk.
  saver.restore(sess, "/tmp/model.ckpt")
  print("Model restored.")
  # Do some work with the model
  ...

### 只保存和读取指定的variables

If you do not pass any argument to tf.train.Saver() the saver handles all variables in the graph. Each one of them is saved under the name that was passed when the variable was created.

注意，不是在save和restore时指定variable，而是在创建Saver时指定variable！！！

因此，你可以创建多个Saver，来保存不同的variable集合。




In [ ]:
# Create some variables.
v1 = tf.Variable(..., name="v1")
v2 = tf.Variable(..., name="v2")
...
# Add ops to save and restore only 'v2' using the name "my_v2"
saver = tf.train.Saver({"my_v2": v2})
# Use the saver object normally after that.
...